# PULSE SWITCHING ANALYSIS
This notebook will find the critical current for current switching loops and will also perform thermal stability fitting.  Estimations of full switching are done by comparison to the user selected *threshold*.

Select file input and threshold below

In [ ]:
# user inputs
dir_path = r'C:\Users\Neuromancer\Desktop\test' # path to directory
file_type = ''
normalize_data = True # change to false to see regular y data, default is true

# Device characteristics
w = 10e-6 # device width (meters)
tau = 10 ** -9 # thermal stability factor
threshold = 0.3 # resistance threshold

In [ ]:
import pandas as pd
import glob
import os
import math
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from scipy import optimize
from AnalysisFunctions import drop_regions, find_zeros, import_datasets, find_resistance_change

all_files = glob.glob(os.path.join(dir_path, '*'+file_type+'*.csv')) # use os.path.join to make os independent
[all_files.remove(x) for x in all_files if 'results' in x] # ignore results file

if len(all_files) != 0:
    full_df = import_datasets(all_files, normalize_data) # if True, data is automatically normalized
    display(full_df.head())
else:
    print(f'No csv files found in {dir_path} with the format: {file_type}!')

#### SELECT DATAFRAME AND GRAPH PARAMETERS:
*x_column* is the column used for x values for graphing and data analysis <br>
*y_column* is the column used for data analysis <br>
*y_normed_column*, if normalize_data is set to true, these values will be the default plot y value <br>
*hue_column* is the column used for seperating the data sets into individual line/scatter plots <br>
*graph_column* is the column that a set of hue_column values is grouped by <br>

The following cell will provide a list of the unique values found in the hue and graph column.

In [ ]:
x_column = 'Current(mA)'
y_column = 'Resistance(Ohm)'
y_normed_column = 'Normalized Resistance(Ohm)'
hue_column = 'Pulse width s' # column to determine how lines should be colored
graph_column = 'Applied in-plane field (Oe)' # seperates data by this column to graph plots (multi-hued)

full_df[hue_column] = full_df[hue_column].apply(lambda x: round(x, 2)) # round to two decimal places
full_df[graph_column] = full_df[graph_column].apply(lambda x: round(x, 2))

# sorted lists of unique values
hue_column_list = np.sort(full_df[hue_column].unique())
graph_column_list = np.sort(full_df[graph_column].unique())

print(hue_column, 'list values in the dataframe: ' + np.array2string(hue_column_list, separator=','))
print(graph_column, 'list values in the dataframe: ' + np.array2string(graph_column_list, separator=','))

In [ ]:
# plot data
sns.set_style('whitegrid')
sns.set_palette('bright', len(hue_column_list))
f = sns.FacetGrid(full_df, hue=hue_column, col=graph_column, height=7, despine=False)
f.map(plt.plot, x_column, y_normed_column).add_legend()

#### IGNORING DATA

Should the user wish to ignore any datasets in the following analysis, including the *graph_column*  value : *hue_column* value in the *ignore_dict* in the following cell will drop the said data from the analysis.  The *graph column* should be the key string and the *hue_column* values should be in a list as ints or floats.  To ignore an *graph_column* value, use *hue_column_list* to ignore all data for said value.

In [ ]:
# fill in dictionary with values to be ignored in the subsequent data processing
ignore_dict = {
    # use format of dictionary key == graph_column value with values == list(hue_column values to ignore)
    '100.0': [0.3],
}
cleaned_df = full_df.copy() # make copy so original import does not need to be repeated
# drop the areas from the full dataframe specified in the ignore_dict
cleaned_df, ig_df, not_found = drop_regions(cleaned_df, ignore_dict, graph_column_list, graph_column, hue_column)
# update list unique values   
hue_column_list = np.sort(cleaned_df[hue_column].unique())
graph_column_list = np.sort(cleaned_df[graph_column].unique())

for string in not_found:
    print(string)

# if there was ignored data, plot said data
if type(ig_df) == pd.DataFrame:
    f = sns.FacetGrid(ig_df, hue=hue_column, col=graph_column, height=7, despine=False)
    f.map(plt.plot, x_column, y_column).add_legend()

#### DATA ANALYSIS:

Pulse Switching Data analysis checks for the critical current values closest to the middle of the loop with the lowest index and will also check the total resistance change and compare it to the *threshold* value determined by the user in the first cell.

By changing the value of *check_left_right* the user can change how many points before and after a zero intercept should be of the same sign.  Loops that have more than two zeros will automatically be flagged and the user will be prompted whether or not to ignore those loops.

When checking the change in resistance, use the *data_percent* variable to select the percentage (as an int) of the data nearest min/max x values to compare. Note that due to there being two y points for each x value, selecting 15 percent will result in a total of **60** percent of the data being compared.

In [ ]:
check_left_right = 1 # number of datapoints to compare
data_percent = 15 # percent of data to check near min and max values
# building the df to the proper size optimizes speed, will be larger than needed if data is ignored
coercivity_df = (pd.DataFrame(index = np.arange(len(graph_column_list) * len(hue_column_list)), 
                              columns=['Positive Critical Current', 
                                       'Negative Critical Current', 
                                       'Average Critical Current',
                                       'Y Zeros',
                                       'Total Resistance Change',
                                       'Resist Change % to Threshold',
                                       '3+ Zeros',
                                       hue_column, 
                                       graph_column],
                             dtype='float')
                )

# go through cleaned_df and find coercivity values (x data) and save in coercivity_df
len_index = 0
multi_zero = {}
for g in graph_column_list:
    # h_list is the specific hue values for each graph value, i.e. skips all values that are ignored
    h_list = np.sort(cleaned_df[(cleaned_df[graph_column] == g)][hue_column].unique())
    for h in h_list:
        # pass in normalized xy values as array for specific loop (graph and hue)
        z, flag = find_zeros(cleaned_df[(cleaned_df[graph_column] == g) & (cleaned_df[hue_column] == h)].
                                       loc[:, [x_column, y_normed_column]].to_numpy(), 
                                       check_left_right
                                      )
        # z1, z2 correspond to the x values where y = 0
        delta_r = find_resistance_change(cleaned_df[(cleaned_df[graph_column] == g) 
                    & (cleaned_df[hue_column] == h)].loc[:, [x_column, y_column]].to_numpy(), data_percent
                                        )
        if flag == 'True':
            if str(g) in multi_zero:
                multi_zero[str(g)].append(h)
            else:
                multi_zero[str(g)] = [h]
        if len(z) != 2:
            print(f'Coercivity values not found for dataset {hue_column} {h} with {graph_column} {g}')
        else:
             coercivity_df.loc[len_index] = z + [(z[0] + z[1])/2, 0.0, delta_r, (delta_r/threshold * 100), flag, h, g]
        len_index += 1

if len(multi_zero) != 0:
    for key, value in multi_zero.items():
        print(f'Multiple zeros found for loop: {key} {value}')
    q = input('Do you wish to ingore data with multiple zeros? (y/n)')
    if q == 'Y' or q == 'y':
        cleaned_df, ig, nf = drop_regions(cleaned_df, multi_zero, graph_column_list, graph_column, hue_column)
        coercivity_df, ig, nf = drop_regions(coercivity_df, multi_zero, graph_column_list, graph_column, hue_column)
        hue_column_list = np.sort(cleaned_df[hue_column].unique())
        graph_column_list = np.sort(cleaned_df[graph_column].unique())
    else:
        pass
else:
    print('Only two zeros detected per loop')
    
coercivity_df.head()

In [ ]:
# plot data with found points
f = sns.FacetGrid(cleaned_df, hue=hue_column, col=graph_column, height=7)
f.map(plt.plot, cleaned_df.columns.values[0], cleaned_df.columns.values[2], zorder=0).add_legend()
for index, ax in enumerate(f.axes[0]):
    ax.scatter(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].iloc[:,0].to_numpy(), 
                       coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].iloc[:,3].to_numpy(), 
                       c='black', marker='D', zorder=1)
    ax.scatter(coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].iloc[:,1].to_numpy(), 
                       coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].iloc[:,3].to_numpy(),
                      c='black', marker='D', zorder=1)

#### SAVE CRITICAL CURRENT and THRESHOLD DATA
Selecting a filename and any specific datasets to ignore the following cell will save a csv with the change in resistance, hue_column and graph_column values.  Thermal stability data can be saved later on.

In [ ]:
filename = 'Icritical'
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
# fill in dictionary with values to be ignored in the subsequent data processing
save_ignore_dict = {
    # use format of dictionary key == graph_column value with values == list(hue_column values to ignore)
    '250.0': [-0.3],
}

threshold_save_df = coercivity_df.copy() # make copy so original import does not need to be repeated
# drop the areas from the full dataframe specified in the ignore_dict
threshold_save_df_df, ig_df, not_found = drop_regions(cleaned_df, save_ignore_dict, graph_column_list, graph_column, hue_column)

for string in not_found:
    print(string)

try:
    threshold_save_df.to_csv(os.path.join(dir_path, file_type + filename + timestamp + 'results.csv'), 
                             encoding='utf-8', index=False)
    print(os.path.join(dir_path, file_type + filename + timestamp + 
                       'results.csv') + ' saved successfully')
except:
    print('Failed to save results.')

#### THERMAL STABILITY FITTING
Using the critical currents found in the previous cells, linear fitting of the upper and lower critical current values is done and the thermal stability  is calculated and stored in a DataFrame.  These results can be graphed and saved in the following cells. 

In [ ]:
# dataframe containing data that will be saved to final output
results_df = (pd.DataFrame(index = np.arange(len(graph_column_list)), 
                           columns=['Upper Fitting Slope', 
                                    'Upper Fitting Intercept',
                                    'Lower Fitting Slope',
                                    'Lower Fitting Intercept',
                                    'Upper Thermal Stability', 
                                    'Lower Thermal Stability',
                                    'Average Thermal Stability',
                                    graph_column])
             )

# use scipy fitting useing non-linear least squares fit
def linear_test_function(x, m, b):
    return (m * x) + b

data_index = 0
for g in graph_column_list:
    try:
        x_vals = np.log(coercivity_df[(coercivity_df[graph_column] == g)][hue_column]/tau)
        u_params, u_params_covariance = (optimize.curve_fit(linear_test_function, # function to test
                                x_vals.values, # x values
                                coercivity_df[(coercivity_df[graph_column] == g)]['Positive Critical Current'].to_numpy()) 
                                         # y values
                                )
        l_params, l_params_covariance = (optimize.curve_fit(linear_test_function, # function to test
                                x_vals.values, # x values
                                coercivity_df[(coercivity_df[graph_column] == g)]['Negative Critical Current'].to_numpy())  
                                         # y values
                                        )
        results_df.loc[data_index] = [u_params[0], u_params[1], l_params[0], l_params[1], 
                                      -(u_params[1]/u_params[0]), -(l_params[1]/l_params[0]), 
                                      ((-(u_params[1]/u_params[0]) + -(l_params[1]/l_params[0]))/2), g]                             
        data_index += 1
    except:
        print(f"An error occured with {g} {graph_column}")

results_df.head()

In [ ]:
# plot data with found coercivities and fitting lines
f = sns.FacetGrid(coercivity_df, col=graph_column, height=7)
coercivity_df[hue_column] = np.log(coercivity_df[hue_column]/tau)
f.map(plt.scatter, hue_column, 'Positive Critical Current')
for index, ax in enumerate(f.axes[0]):
    x_val = coercivity_df[(coercivity_df[graph_column] == graph_column_list[index])].loc[:, hue_column]
    ax.scatter(x_val.values, coercivity_df[(coercivity_df[graph_column] == 
                    graph_column_list[index])].loc[:,'Negative Critical Current'].values, color='orange')
    ax.plot(x_val, linear_test_function(x_val.values, *[
                    results_df[(results_df[graph_column] == graph_column_list[index])]['Upper Fitting Slope'][0], 
                    results_df[(results_df[graph_column] == graph_column_list[index])]['Upper Fitting Intercept'][0]]), 
                    color='blue')
    ax.plot(x_val, linear_test_function(x_val.values, *[
                results_df[(results_df[graph_column] == graph_column_list[index])]['Lower Fitting Slope'][0], 
                results_df[(results_df[graph_column] == graph_column_list[index])]['Lower Fitting Intercept'][0]]),
                color='orange')
    ax.set_ylabel('Critical Current')
    ax.set_xlabel('Natural Log of Pulse Width / Tau')
    

In [ ]:
filename = 'ThermalStability'
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
save_ignore = [
    # list of values to ignore go here
    # 100,
    # -1000,
]

for n in save_ignore:
    try:
        results_df = results_df.drop(results_df[(results_df[graph_column] == n)].index, inplace = True)
        print(f'Dropped value {n} from the results dataframe')
    except:
        print(f'Failed to drop value {n}!')

try:
    results_df.to_csv(os.path.join(dir_path, file_type + filename + timestamp + 'results.csv'), encoding='utf-8', index=False)
    print(os.path.join(dir_path, file_type + filename + timestamp + 'results.csv') + ' saved successfully')
except:
    print('Failed to save results.')